# 正規表現についての課題

正規表現は使いこなせば大変効率的なプログラムを作れる可能性があります．
その反面，思わぬ落とし穴があっても見過ごしてしまう危険もあります．
適切な正規表現を作成するには，多くのテストケースを作成して確認することです．

ここでは，ここまで学習したことを利用して正規表現の課題を実施してください．
そして，正規表現の利用では思ってもいなかった結果になる危険性があることを学んでください．

## 課題

任意の文章の中に記載されている<font color=blue>正の整数</font>を全て数値として抽出するプログラムを正規表現を用いて作成してください．

【条件】
- 抽出数値は整数のみとします．
- 整数は文章の中に複数存在する可能性があります．
- 整数は半角文字とします．
- 整数は3桁ごとにカンマ区切りの場合とカンマ無しの場合があります．
    - 例「123,456,789」，「12,345」，「12345」
- カンマの位置が間違っている数値は無効とします．
- 複数の整数をカンマと空白を用いて続けて書くことがあります．
    - 例「12, 13, 14, 15」
- プラスマイナス記号は無視します．
- 小数点付数は抽出から除外します．

（注）1個の正規表現で完結しようとすると思わぬバグが出る危険性があります．
確実で分かりやすいプログラムを心がけてください．

## アウトライン

次の手順でプログラミングを行って行きます．

1. テスト用の文章を用意します．
- reライブラリーを搬入します．
- 正規表現パターンの文字列を作成して，プログラムを作成します．
    - 正規表現は単純な表現から徐々に複雑な表現に拡張していきます．
    - 文章から全ての数値の抽出には，re.findall()関数を使用します．
    - 一つの正規表現だと難しい場合は，複数のステップに分けることを検討します．
- テストが完了したら，パターンをコンパイルします．
    - コンパイル済みオブジェクトで最終確認をします．

## テスト用文章

テスト用文章として下記のデータを用意します．

```Python
string1 = '主催者側の発表では13,450となっていますが，当局の集計では8,234となっています．'
string2 = '表現は異なりますが，12345と12,345は同じ値です．'
string3 = '桁数の違いによって1や12,345さらに1,234,567および12,345,678,900なども対象になります．'
string4 = '数字は三桁ごとに区切るので12,34や1,2345,6などは間違いです．'
string5 = '123は有理数123.45の整数部です．'
string6 = '合格者の番号は「4, 56, 138, 260, 379」です．'
```

In [1]:
string1 = '主催者側の発表では13,450となっていますが，当局の集計では8,234となっています．'
string2 = '表現は異なりますが，12345と12,345は同じ値です．'
string3 = '桁数の違いによって1や12,345さらに1,234,567および12,345,678,900なども対象になります．'
string4 = '数字は三桁ごとに区切るので12,34や1,2345,6などは間違いです．'
string5 = '123は有理数123.45の整数部です．'
string6 = '合格者の番号は「4, 56, 138, 260, 379」です．'

*****
## 課題の解答

まず，reライブラリーを搬入します．

```Python
import re
```

In [2]:
import re

### 単純な正の整数
カンマ付き整数を表す正規表現を順番に構成していきます．
既に学習した整数の表現を土台として拡張します．
単純な正の整数は次の表現で表します．

>  
```Python
r'\d+'
```

この正規表現はカンマで区切らない整数です．
これでカンマ区切りのある文章の検索をするとどうなるか，確認します．

```Python
re.findall(r'\d+',string1)
```

In [3]:
re.findall(r'\d+',string1)

['13', '450', '8', '234']

このように，カンマで区切られた部分を独立の数値と認識して抽出してしまいます．

### カンマ区切りの正の整数

まず，カンマ区切りの正の整数の実際の文字列を確認します．

| 数値 | 一致するパターン |
|:---:|:---:|
| 1~999 | &yen;d{1,3} |
| 1,000~999,999 | &yen;d{1,3},&yen;d{3} |
| 1,000,000~999,999,999 | &yen;d{1,3},&yen;d{3},&yen;d{3} |


このように桁を増やしていくと，文字列が「&yen;d{1,3}」で始まって，その後に「,&yen;d{3}」を繰返す形になります．

> 
```Python
r'\d{1,3}(,\d{3})*'
```

このパターンによる抽出をテストします．

```Python
re.findall(r'(\d{1,3}(,\d{3})*)',string1)
```

In [4]:
re.findall(r'(\d{1,3}(,\d{3})*)',string1)

[('13,450', ',450'), ('8,234', ',234')]

テスト文書 string1 については，正しく抽出できています．
次に string2 について確認します．

```Python
re.findall(r'(\d{1,3}(,\d{3})*)',string2)
```

In [5]:
re.findall(r'(\d{1,3}(,\d{3})*)',string2)

[('123', ''), ('45', ''), ('12,345', ',345')]

この結果として，数値'12345'が二つに分割されて検出されてしまっています．
このパターン検索では上手くいきません．
そこで，カンマ無し数値にも反応するように両方のパターンを含んだ正規表現を考えます．

>  
```Python
r'(\d+)|(\d{1,3}(,\d{3})*)'
```

これは，カンマ無しの正の整数<font face='courier new' color=blue>&yen;d+</font>と
カンマ区切りの整数<font face='courier new' color=blue>&yen;d{1,3}(,&yen;d{3})\*</font>の
どちらかにマッチする正規表現です．

この正規表現でテストします．

```Python
re.findall(r'(\d+)|(\d{1,3}(,\d{3})*)',string2) 
```

In [6]:
re.findall(r'(\d+)|(\d{1,3}(,\d{3})*)',string2) 

[('12345', '', ''), ('12', '', ''), ('345', '', '')]

今度はカンマ無しの数値は正しく抽出されましたが，カンマ付きの数値「12,345」が 12 と 345 に分割されてしまいました．
この正規表現も上手くいきません．

### 二段階処理

ここまで見てきたように，一つの正規表現を複雑化して一度の処理で目的の数値を取り出す方法には副作用があります．
そこで，処理を二段階に分けて行う方法を検討します．

一段階目の処理は，とにかく数値らしき文字列だけを抽出したリスト配列を作ります．
この処理にはre.findall()関数を使用します．
そこで数値とピリオドおよびカンマを含む一連の文字列を次の正規表現で抽出します．

> 
```Python
r'[\d,.]*\d'
```

re.findall()関数でこの正規表現を使うことによって，数値らしき文字列が全てリスト配列となって抽出されます．

```Python
re.findall(r'[\d,.]*\d',string2)
```

In [7]:
re.findall(r'[\d,.]*\d',string2)

['12345', '12,345']

次に，取り出したリスト配列の各要素が私たちが想定している正の整数にマッチするかをre.fullmatch()関数で確認して抽出します．
そこで使用する正規表現は，次のようになります．

>  
```Python
r'\d{1,3}(,\d{3})*|\d+'
```

取り出した数字の列に対して，このパターンの完全一致を見るので，文章中で発生したカンマ付きとカンマ無しのトラブルを回避することができます．
第一段階で取り出したリスト配列['12345', '12,345']に対してリスト内包表記によってre.fullmatch()を実行します．

```Python
[x for x in ['12345', '12,345'] if re.fullmatch(r'\d{1,3}(,\d{3})*|\d+',x) ]
```

In [8]:
[x for x in ['12345', '12,345'] if re.fullmatch(r'\d{1,3}(,\d{3})*|\d+',x) ]

['12345', '12,345']

このようにプログラムを二段階にすることによって正の整数を正しく抽出できるようになりました．


## コンパイル

それでは，全てのテスト用文章で確認するために，オーバーヘッドとなるコンパイルを事前に実施することにします．

```Python
step1 = re.compile(r'[\d,.]*\d')
step2 = re.compile(r'\d{1,3}(,\d{3})*|\d+')
```

In [9]:
step1 = re.compile(r'[\d,.]*\d')
step2 = re.compile(r'\d{1,3}(,\d{3})*|\d+')

## テスト用関数

それぞれの文章に対して数値を取り出す処理は同じプログラムになるので，これを関数として定義します．  
抽出した数字を数値化する前に，<font color=green>replace(r',','')</font>によってカンマを削除しています．  
このプログラムでは次の3つの値をprint()関数で表示します．
- 対象の文章
- 取り出した数字の塊
- 正の整数を数値化した値

```Python
def testProgram(string):
    print('文章→',string)
    temp = (step1.findall(string))
    print('抽出→',temp)
    print('数値→',[int(x.replace(r',','')) for x in temp if step2.fullmatch(x)])
```

In [10]:
def testProgram(string):
    print('文章→',string)
    temp = (step1.findall(string))
    print('抽出→',temp)
    print('数値→',[int(x.replace(r',','')) for x in temp if step2.fullmatch(x)])

## テストの実施

それぞれの文章にテストプログラムを適用して正しく動作する事を確認します．

In [11]:
testProgram(string1)

文章→ 主催者側の発表では13,450となっていますが，当局の集計では8,234となっています．
抽出→ ['13,450', '8,234']
数値→ [13450, 8234]


In [12]:
testProgram(string2)

文章→ 表現は異なりますが，12345と12,345は同じ値です．
抽出→ ['12345', '12,345']
数値→ [12345, 12345]


In [13]:
testProgram(string3)

文章→ 桁数の違いによって1や12,345さらに1,234,567および12,345,678,900なども対象になります．
抽出→ ['1', '12,345', '1,234,567', '12,345,678,900']
数値→ [1, 12345, 1234567, 12345678900]


In [14]:
testProgram(string4)

文章→ 数字は三桁ごとに区切るので12,34や1,2345,6などは間違いです．
抽出→ ['12,34', '1,2345,6']
数値→ []


In [15]:
testProgram(string5)

文章→ 123は有理数123.45の整数部です．
抽出→ ['123', '123.45']
数値→ [123]


In [16]:
testProgram(string6)

文章→ 合格者の番号は「4, 56, 138, 260, 379」です．
抽出→ ['4', '56', '138', '260', '379']
数値→ [4, 56, 138, 260, 379]


以上の結果を見ると，正の整数を抽出するプログラムが正しく機能していることが分かります．

## スリム化した関数の定義

テスト用のプログラムは途中結果を出力するために冗長なプログラムにしていました．
そこで，単に結果を返す関数にします．

```Python
def extractNumericalValues(string):
    return [int(x.replace(r',','')) for x in step1.findall(string) if step2.fullmatch(x)]
```

In [17]:
def extractNumericalValues(string):
    return [int(x.replace(r',','')) for x in step1.findall(string) if step2.fullmatch(x)]

念のためにテスト文章からの抽出を実施します．
テスト文章の下に抽出した正の整数を表示します．

In [18]:
print('*****\n',string1,'\n',extractNumericalValues(string1))
print('*****\n',string2,'\n',extractNumericalValues(string2))
print('*****\n',string3,'\n',extractNumericalValues(string3))
print('*****\n',string4,'\n',extractNumericalValues(string4))
print('*****\n',string5,'\n',extractNumericalValues(string5))
print('*****\n',string6,'\n',extractNumericalValues(string6))

*****
 主催者側の発表では13,450となっていますが，当局の集計では8,234となっています． 
 [13450, 8234]
*****
 表現は異なりますが，12345と12,345は同じ値です． 
 [12345, 12345]
*****
 桁数の違いによって1や12,345さらに1,234,567および12,345,678,900なども対象になります． 
 [1, 12345, 1234567, 12345678900]
*****
 数字は三桁ごとに区切るので12,34や1,2345,6などは間違いです． 
 []
*****
 123は有理数123.45の整数部です． 
 [123]
*****
 合格者の番号は「4, 56, 138, 260, 379」です． 
 [4, 56, 138, 260, 379]


最終的なプログラムは次のようになりました．

>  
```Python
step1 = re.compile(r'[\d,.]*\d')
step2 = re.compile(r'\d{1,3}(,\d{3})*|\d+')
def extractNumericalValues(string):
    return [int(x.replace(r',','')) for x in step1.findall(string) if step2.fullmatch(x)]
```

これで，課題の解答を終了します．

*****